In [1]:
from __future__ import print_function, division, absolute_import
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))
# from clr_callback import CyclicLR
# import dill
from BalancedDannAudioDataGenerator import BalancedAudioDataGenerator
import os
from scipy.io import loadmat
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
import math
import pandas as pd
import tables,h5py
from datetime import datetime
import argparse
from keras.callbacks import TensorBoard, Callback, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger
from keras import backend as K
from keras.utils import plot_model
from Heartnet import heartnet 
from dann_heartnet_v1 import log_macc, write_meta, compute_weight, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns

import dataLoader
sns.set()

Using TensorFlow backend.


In [2]:
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/folds_phys_compare_pascal/'

In [23]:
train_domains = 'abcd'
test_domains = 'fgi'
test_split = 0

num_class_domain = len(set(train_domains + test_domains))
num_class = 2

x_train, y_train, y_domain, train_parts = dataLoader.getData(fold_dir,train_domains)
x_val, y_val, val_domain, val_parts = dataLoader.getData(fold_dir,test_domains)
val_files = val_domain
#Create meta labels and domain labels
domainClass = [(cls,dfc) for cls in range(2) for dfc in train_domains]
meta_labels = [domainClass.index((cl,df)) for (cl,df) in zip(y_train,y_domain)]

y_domain = np.array([list(set(train_domains+test_domains)).index(lab) for lab in y_domain])
val_domain = np.array([list(set(train_domains+test_domains)).index(lab) for lab in val_domain])

################### Reshaping ############
[x_train, x_val], [y_train,y_domain,y_val] = reshape_folds([x_train,x_val],[y_train,y_domain,y_val])
y_train = to_categorical(y_train, num_classes=num_class)
y_domain = to_categorical(y_domain,num_classes=num_class_domain)
y_val = to_categorical(y_val, num_classes=num_class)
val_domain = to_categorical(val_domain,num_classes=num_class_domain)
print("Train  files ", y_train.shape, "  Domain ", y_domain.shape)

(19734, 2500, 1)
(21761, 2500, 1)
(19734, 1)
(19734, 1)
(21761, 1)
Train  files  (19734, 2)   Domain  (19734, 7)


In [5]:
datagen = BalancedAudioDataGenerator(shift=.1)
flow = datagen.flow(x_train, [y_train,y_domain],
                            meta_label=meta_labels,
                            batch_size=1020, shuffle=True,
                            seed=2)

Chunk size selected as 127


/media/mhealthra2/Data/heart_sound/Adversarial-Heart-Sound-Classification/codes/BalancedDannAudioDataGenerator.py:823: UserWarning: `meta_labels` specified, will use meta_labels instead of target_label
  warnings.warn('`meta_labels` specified, will use meta_labels instead of target_label')


In [8]:
wow = flow.next()

In [19]:
type(wow[0]),type(wow[1][0]),type(wow[1][1])

(numpy.ndarray, numpy.ndarray, numpy.ndarray, 2)

In [20]:
wow[0].shape,wow[1][0].shape,wow[1][1].shape

((1016, 2500, 1), (1016, 2), (1016, 7))

In [40]:
hey = []
for cls in range(2):
    for dfc in 'abcdg':
        hey.append((cls,dfc))

In [43]:
domainClass = [(cls,dfc) for cls in range(2) for dfc in train_domain]
meta_label = [hey.index((cl,df)) for (cl,df) in zip(y,yd)]

In [69]:
np.array([list(set('abcd'+'ef')).index(lab) for lab in yd])

4

In [21]:
list(set('sdff'+'cdc'))

['f', 's', 'c', 'd']

In [54]:
y[18000:18100],yd[18000:18100],meta_label[18000:18100]

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 ['c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c',
  'c

In [26]:
max(np.bincount(meta_labels))

9864

In [55]:
hey

[(0, 'a'),
 (0, 'b'),
 (0, 'c'),
 (0, 'd'),
 (0, 'g'),
 (1, 'a'),
 (1, 'b'),
 (1, 'c'),
 (1, 'd'),
 (1, 'g')]